In [ ]:
import os
import sys

os.chdir("..")
sys.path.append("../../")

# Plot for visualization

## Introduction

In this section, we will show how to use function `plot_reduceddim()` to realize dimension reduction and automatically plot the result.

To get detailed information of the input and output of the function, please check [API](../set_up/_autosummary/pyscDesign3.plot_reduceddim.rst).

## Step 1: Import packages and Read in data

### import packages

In [ ]:
import anndata as ad
import numpy as np
import pandas as pd
import pyscDesign3

### Read in data

The raw data is from the [scvelo](https://scvelo.readthedocs.io/scvelo.datasets.pancreas/) and we only choose top 30 genes to save time.

In [ ]:
data = ad.read_h5ad("data/PANCREAS.h5ad")
data = data[:, 0:30]
data

## Step 2: Use `scDesign3` class to simulate reads for comparison

In [ ]:
# create the instance and set the parallel method
test = pyscDesign3.scDesign3(n_cores=6, parallelization="mcmapply", return_py=True)
test.set_r_random_seed(123)

# all-in-one simulation
simu_res = test.scdesign3(
    anndata=data,
    default_assay_name="counts",
    celltype="cell_type",
    pseudotime="pseudotime",
    mu_formula="s(pseudotime, k = 10, bs = 'cr')",
    sigma_formula="s(pseudotime, k = 5, bs = 'cr')",
    family_use="nb",
    usebam=True,
    corr_formula="1",
    copula="gaussian",
)

## Step 3: Use the simulated counts to construct the new `anndata.AnnData` object

Besides constructing the simulated `anndata.AnnData` object, we can also calculate the log transformed data for visualization.

In [ ]:
simu_data = ad.AnnData(X=simu_res["new_count"], obs=simu_res["new_covariate"])
simu_data.layers["log_transformed"] = np.log1p(simu_data.X)
data.layers["log_transformed"] = np.log1p(data.X)

## Step 4: Visualization

Please make sure all `anndata.AnnData` objects provided in `ref_anndata` and `anndata_list` has similar assay name structure.

For example, all involved count matrix should all be stored either in `anndata.AnnData.X` or `anndata.AnnData.layers[assay_use]` and all share the same name.

In [ ]:
plot = pyscDesign3.plot_reduceddim(
    ref_anndata=data,
    anndata_list=simu_data,
    name_list=["Reference", "scDesign3"],
    assay_use="log_transformed",
    if_plot=True,
    color_by="pseudotime",
    n_pc=20,
    point_size=5,
)

If you set `if_plot` = True, then the return result is a `dict` with two keys: **p_umap** and **p_pca**. Each key hosts a plot plotted by `matplotlib`.

UMAP plot

In [ ]:
plot["p_umap"]

PCA plot

In [ ]:
plot["p_pca"]

## Alternative to plot

If you are not satisfied with the automatically generated plot result, you can set `if_plot` = False. Then you will get the dimension reduction result and customize your manipulation.

In [ ]:
res = pyscDesign3.plot_reduceddim(
    ref_anndata=data,
    anndata_list=simu_data,
    name_list=["Reference", "scDesign3"],
    assay_use="log_transformed",
    if_plot=False,
    color_by="pseudotime",
    n_pc=20,
    point_size=5,
)

Each row of the output represents a cell (observation). The return PCs number is equal to the set in `n_pc`.

In [ ]:
res[["Method","pseudotime","PC1","PC2","UMAP1","UMAP2"]].head()